# Extraction notebook

This notebook is responsible for the extraction of Sales tables from AdventureWorks database.

In [ ]:
%pip install pandas SQLAlchemy==1.4.54



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import sqlalchemy 
from sqlalchemy import create_engine
from pyspark.sql import SparkSession

✅ Bibliotecas importadas com sucesso!


In [4]:
# Recuperando credenciais armazenadas no Databricks Secrets
host = dbutils.secrets.get(scope="mateus_marin_adw", key="host")
database = dbutils.secrets.get(scope="mateus_marin_adw", key="database")
login = dbutils.secrets.get(scope="mateus_marin_adw", key="login")
senha = dbutils.secrets.get(scope="mateus_marin_adw", key="senha")
port = dbutils.secrets.get(scope="mateus_marin_adw", key="port")

# Escolhendo o catálogo RAW onde os dados serão armazenados
catalog_raw = "mateus_marin_raw"

print(f"Credenciais recuperadas e catálogo definido: {catalog_raw}")

dbutils.widgets.text("managed_location", "default_value", "Managed Location")
managed_location = dbutils.widgets.get("managed_location")

Credenciais recuperadas e catálogo definido: mateus_marin_raw


/mnt/c/Users/Mateus/Documents/indicium/lighthouse/desafio-final-lh/.venv/lib/python3.11/site-packages/databricks/sdk/_widgets/__init__.py:61: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [10]:
# Criando URL de conexão com JDBC
url_job = f"jdbc:sqlserver://{host}:{port};databaseName={database};encrypt=false"

# Configurações de conexão
connection_properties = {
    "user": login,
    "password": senha,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

print("Configuração de conexão definida!")

Configuração de conexão definida!


In [11]:
# Query para listar todas as tabelas do schema `Sales`
query = """
SELECT TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'Sales' AND TABLE_TYPE = 'BASE TABLE'
"""

try:
    # Lendo as tabelas do banco via JDBC
    df_tables = spark.read.jdbc(url=url_job, table=f"({query}) as sales_tables", properties=connection_properties)

    # Corrigindo a extração sem usar `.rdd()`
    sales_tables = [row.TABLE_NAME for row in df_tables.collect()]  

    print(f"Tabelas encontradas no schema Sales: {sales_tables}")

except Exception as e:
    print(f"Erro ao recuperar tabelas: {e}")
    sales_tables = []  # Evita erro no loop

Tabelas encontradas no schema Sales: ['SalesTaxRate', 'PersonCreditCard', 'SalesTerritory', 'SalesTerritoryHistory', 'ShoppingCartItem', 'SpecialOffer', 'SpecialOfferProduct', 'Store', 'CountryRegionCurrency', 'CreditCard', 'Currency', 'CurrencyRate', 'Customer', 'SalesOrderDetail', 'SalesOrderHeader', 'SalesOrderHeaderSalesReason', 'SalesPerson', 'SalesPersonQuotaHistory', 'SalesReason']


In [12]:
# Nome do catálogo Unity Catalog
catalog_raw = "mateus_marin_raw"

def extract_table(table_name):
    try:
        print(f"Extraindo tabela: {table_name}...")

        # Criando DataFrame Spark a partir do SQL Server via JDBC
        df = spark.read.jdbc(url=url_job, table=f"Sales.{table_name}", properties=connection_properties)

        # Nome da tabela no Unity Catalog
        table_path = f"{catalog_raw}.sales.{table_name.lower()}"

        # Salvando no formato Delta Table no Unity Catalog
        df.write.format("delta").mode("overwrite").saveAsTable(table_path)

        print(f"Tabela {table_name} carregada com sucesso em {table_path}!")

    except Exception as e:
        print(f"Erro ao extrair {table_name}: {e}")
        return False  # Retorna False se falhar

    return True  # Retorna True se for bem-sucedida

In [13]:
# Executar a extração para todas as tabelas do schema `Sales`
failed_tables = []  # Lista para armazenar tabelas que falharam
successful_tables = []  # Lista para armazenar tabelas extraídas com sucesso

for table in sales_tables:
    result = extract_table(table)
    if result:
        successful_tables.append(table)
    else:
        failed_tables.append(table)

# Exibir relatório final
print("\n**Resumo da Extração**")
print(f"Tabelas extraídas com sucesso: {successful_tables}" if successful_tables else "Nenhuma tabela foi extraída com sucesso.")
print(f"Tabelas que falharam: {failed_tables}" if failed_tables else "Todas as tabelas foram extraídas corretamente.")

Extraindo tabela: SalesTaxRate...
Tabela SalesTaxRate carregada com sucesso em mateus_marin_raw.sales.salestaxrate!
Extraindo tabela: PersonCreditCard...
Tabela PersonCreditCard carregada com sucesso em mateus_marin_raw.sales.personcreditcard!
Extraindo tabela: SalesTerritory...
Tabela SalesTerritory carregada com sucesso em mateus_marin_raw.sales.salesterritory!
Extraindo tabela: SalesTerritoryHistory...
Tabela SalesTerritoryHistory carregada com sucesso em mateus_marin_raw.sales.salesterritoryhistory!
Extraindo tabela: ShoppingCartItem...
Tabela ShoppingCartItem carregada com sucesso em mateus_marin_raw.sales.shoppingcartitem!
Extraindo tabela: SpecialOffer...
Tabela SpecialOffer carregada com sucesso em mateus_marin_raw.sales.specialoffer!
Extraindo tabela: SpecialOfferProduct...
Tabela SpecialOfferProduct carregada com sucesso em mateus_marin_raw.sales.specialofferproduct!
Extraindo tabela: Store...
Tabela Store carregada com sucesso em mateus_marin_raw.sales.store!
Extraindo tabel